# Merging and Clipping

> Making use of H3 global grid indexing when working with rasters. For this example series, we focus on B04 (red), B03 (green), B02 (blue), and B08 (nir).

---
__Last Update:__ 18 JAN 2024 [Mosaic 0.3.14]

## Setup Notes

> Raster processing is memory intensize. Here are some config tips.

* This notebook was run on AWS [r5d.4xlarge](https://www.databricks.com/product/pricing/product-pricing/instance-types) instances (1-10 workers auto-scaling for up to 160 concurrent tasks).
* __Optional:__ Prior to launching, the following can be added to the cluster spark configs, adapted from this databricks [blog](https://www.databricks.com/blog/2015/05/28/tuning-java-garbage-collection-for-spark-applications.html) and these JVM tuning [docs](https://docs.oracle.com/cd/E40972_01/doc.70/e40973/cnf_jvmgc.htm#autoId2): `spark.executor.extraJavaOptions -XX:+UseG1GC -XX:+PrintFlagsFinal -XX:InitiatingHeapOccupancyPercent=35 -XX:G1HeapRegionSize=32M -XX:ParallelGCThreads=16 -XX:ConcGCThreads=4`. _This optimization is not required for this example._

## Imports + Config

In [0]:
%pip install --quiet 'databricks-mosaic<0.4,>=0.3'
%pip install --quiet rasterio==1.3.5 gdal==3.4.3 pystac pystac_client planetary_computer tenacity rich pandas==1.5.3

In [0]:
# -- configure AQE for more compute heavy operations
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.shuffle.partitions", 512)

# -- import databricks + delta + spark functions
from delta.tables import *
from pyspark.databricks.sql import functions as dbf
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

# -- setup mosaic
import mosaic as mos

mos.enable_mosaic(spark, dbutils)
mos.enable_gdal(spark)

# -- other imports
from datetime import datetime
from io import BytesIO
from matplotlib import pyplot
from rasterio.io import MemoryFile

import library
import os
import pathlib
import planetary_computer
import pystac_client
import rasterio
import requests
import warnings

warnings.simplefilter("ignore")

In [0]:
mos.__version__

In [0]:
%reload_ext autoreload
%autoreload 2
%reload_ext library

### Databricks Catalog + Schema

> This is for reading and writing out table(s). __Note: these should already exists after running notebook 01.__

In [0]:
# adjust to match your catalog + schema
catalog_name = "geospatial_docs"
db_name = "eo_alaska"

sql(f"""USE CATALOG {catalog_name}""")
sql(f"""USE DATABASE {db_name}""")

## Data load

We can easily browse the data we have downloaded previously (notebook 01). The download metadata is stored as a delta table.

In [0]:
%sql show tables

In [0]:
tile_df = spark.table("band_b02_tile")
h3_df =spark.table("band_b02_h3")

## Merge
We can easily merge tiles using `rst_merge_agg`, from [docs](https://databrickslabs.github.io/mosaic/api/raster-functions.html?highlight=rst_mergeagg#rst_mergeagg):

> Combines a collection of rasters into a single raster. The rasters do not need to have the same extent. The rasters must have the same coordinate reference system. The rasters are combined using gdalwarp. The noData value needs to be initialised; if not, the non valid pixels may introduce artifacts in the output raster. The rasters are stacked in the order they are provided. This order is randomized since this is an aggregation function. If the order of rasters is important please first collect rasters and sort them by metadata information and then use rst_merge function. The output raster will have the extent covering all input rasters. The output raster will have the same number of bands as the input rasters. The output raster will have the same pixel type as the input rasters. The output raster will have the same pixel size as the highest resolution input rasters. The output raster will have the same coordinate reference system as the input rasters.

_Here we merge back an 'item_id' tiles from the same area as looked at previously._

In [0]:
display(
  tile_df
    .filter(col("item_id").contains("_R014_"))
    .groupBy("item_id")
      .count()
    .orderBy(F.desc("count"))
    .limit(25)
)

In [0]:
to_plot = (
  tile_df
    .filter("item_id = 'S2A_MSIL2A_20210601T204021_R014_T08WMS_20210602T071855'")
    .groupBy("item_id")
    .agg(mos.rst_merge_agg("tile").alias("tile"))
  .collect()
)

In [0]:
library.plot_raster(to_plot[0]["tile"]["raster"])

Mosaic framework provides the same tessellation principles for both vector and raster data. We can project both vector and raster data into a unified grid and from there it is very easy to combine and join raster to raster, vector to vector and raster to vector data. _Below we look at a given H3 cellid with multiple tiles within._

_[1] Find index_id(s) with 2+ tiles._

In [0]:
display (
  h3_df
  .groupBy("index_id")
    .count()
  .orderBy(F.desc("count"))
  .limit(25)
)

_[2] Collect list of tiles for the index id(s) chosen._

> Note: breaking down steps for clarity. See that 'tile' column is now a list of tiles for the 'index_id'.

In [0]:
for_plot = (
   h3_df
  .filter("index_id == 608598778412269567")
  .groupBy("index_id")
  .agg(
    F.count(F.lit(1)).alias("cnt"),
    F.collect_list("tile").alias("tile")
  )
  .orderBy(F.desc("cnt"))
)
display(for_plot)

_[3] Call `rst_combineavg` on the collected 'tile' list._

> Note: this replaces the initial 'tile' list column with a single raster which uses the average pixel value for any overlaps.

In [0]:
for_plot1 = for_plot.withColumn("tile", mos.rst_combineavg("tile"))
for_plot1.display()

_[4] Collect to render._

In [0]:
to_plot = for_plot1.collect()

In [0]:
library.plot_raster(to_plot[0]["tile"]["raster"])

## Raster for Arbitrary Corridors

> To illustrate how easy it is to combine vector and raster data we will use a traditionally hard problem. Extraction of raster data for an arbitrary corridors.

__Notes:__

1. Though we specify a WKT for the corridor below, you can pull any any vector data including popular formats such as shapefiles, geodatabases, and geoparquet as well as supported columnar interchange formats (WKT, WKB, GeoJSON). This is covered in other material, but rest assured that you have the entire arsenal of product, DBLabs Mosaic, and external library supplied functions at your disposal. 
1. We use the same H3 resolution of `7` to match our earlier preparation in `h3_df` as the 'index_id' will be the basis of the join.

In [0]:
line_example = "LINESTRING(-158.34445841325555 68.0176784075422,-155.55393106950555 68.0423396963395,-154.82883341325555 67.84431100260183,-159.33322794450555 67.81114172848677,-160.01438028825555 67.47684671455214,-154.43332560075555 67.56925103744871,-154.01584513200555 67.30791374746678,-160.16818888200555 67.25700024664256,-160.58566935075555 66.94924133006975,-153.73020060075555 67.0693906319206,-154.49924356950555 66.70715520513478,-160.12424356950555 66.70715520513478,-159.02561075700555 66.37476822845568,-154.56516153825555 66.49774379983036,-155.04855997575555 66.22462528148408,-158.76193888200555 66.16254082040112,-157.94895060075555 65.94851918639993,-155.64182169450555 66.0021934684043,-158.58615763200555 66.55900493948819,-155.26828653825555 67.43472555587037,-161.64035685075555 67.86087797718164,-161.66232950700555 67.44315575603868)"

line_df = (
  spark.createDataFrame([line_example], "string")
  .select(F.col("value").alias("wkt"))
  .select(
    mos.grid_tessellateexplode("wkt", F.lit(7)).alias("grid")
  )
  .select("grid.*")
)
print(f"count? {line_df.count():,}")
line_df.limit(1).show(vertical=True) # <- show + limiting for ipynb only

[1] We can visualise all the cells of interest for the provided arbitrary corridor. Since we are now operating in grid space it is very easy to get all raster images that match this specification.

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# line_df index_id h3 5_000

[2] We can quickly join on matching H3 'index_ids' to see which rasters match. This can be further enhanced by time filters and multiple bands (not shown here).

In [0]:
cells_of_interest = (
  h3_df
    .join(line_df, on=["index_id"])
)
print(f"count? {cells_of_interest.count():,}")
cells_of_interest.limit(10).display()

[3] We can render the tile counts per H3 'index_id' to better understand areas of matching data.

In [0]:
plot_cells = (
  cells_of_interest
    .groupBy("index_id")
      .count()
)
print(f"count? {plot_cells.count():,}")

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# plot_cells index_id h3 5_000

### Clip to Vector

What if we want rasters for all pixels in the corridor? We can apply `rst_clip`, from [docs](https://databrickslabs.github.io/mosaic/api/raster-functions.html#rst_clip):

> Clips the raster to the geometry. The geometry is expected to be in the same coordinate reference system as the raster. The geometry is expected to be a polygon or a multipolygon. The output raster will have the same extent as the input geometry. The output raster will have the same number of bands as the input raster. The output raster will have the same pixel type as the input raster. The output raster will have the same pixel size as the input raster. The output raster will have the same coordinate reference system as the input raster.

__Note: to get our line corridor into an accepted geometry, we will buffer it by a radius `0.001` units (degrees for 4326) which is 111m at the equator.__ This takes ~1.5 minutes using the notebook cluster.

_[1] We start with a buffered line geometry as basis of clipping._

In [0]:
poly_df = (
  spark.createDataFrame([line_example], "string")
  .select(mos.st_buffer("value", F.lit(0.001)).alias("wkt"))
)
poly_df.display()

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# poly_df wkt geometry

_[2] Approximate clip (at H3 cell id level) is efficient (and should be done first)._

> You might have filtered `h3_df` in some way, here we show just a single band for a single day for brevity.

__Notes:__

1. Use H3 cell ids for both the rasters and the initial corridor as part of filtering to relevant data.
1. We write out a Delta table to help with performance.

In [0]:
clip_h3_df = None
clip_h3_tbl_name = "band_b02_h3_clip"
if not spark.catalog.tableExists(clip_h3_tbl_name):
  (
    poly_df
      .select(mos.grid_tessellateexplode("wkt", F.lit(7)))
      .selectExpr("index.*")
      .write
        .saveAsTable(clip_h3_tbl_name)
  )
sql(f"""OPTIMIZE {clip_h3_tbl_name} ZORDER BY (index_id)""")
clip_h3_df = spark.table(clip_h3_tbl_name)
print(f"count? {clip_h3_df.count():,}")
clip_h3_df.limit(10).display()

In [0]:
for_clip_df = (
   h3_df
    .join(
      clip_h3_df.selectExpr("index_id","is_core as is_core_clip", "wkb as wkb_clip").distinct(),
      ["index_id"]
    )
)
print(f"count? {for_clip_df.count():,}")
for_clip_df.limit(10).display()

_[3] For precise clipping we use `rst_clip` and the H3 tessellate fields._

__Notes:__

1. For this, we use the approximate result first to avoid unnecessary comparisons.
1. We are testing the actual wkb_buffer on the tiles.
1. The results are the intersections.

In [0]:
display(
  for_clip_df
    .selectExpr("tile","cast(srid as integer) as srid","wkb_clip")
    .withColumn("geom_clip", mos.st_setsrid(mos.st_geomfromwkb("wkb_clip"), F.lit(4326)))
    .withColumn(
      "geom_clip",
      mos.st_transform("geom_clip", "srid")
    )
  .limit(10)
)

In [0]:
clip_df = (
  for_clip_df
    .withColumn("geom_clip", mos.st_setsrid(mos.st_geomfromwkb("wkb_clip"), F.lit(4326)))
    .withColumn("geom_clip", mos.st_transform("geom_clip", F.expr("cast(srid as integer)")))
    .withColumn(
      "clipped_tile",
      F
        .when(col("is_core_clip"), col("tile"))
        .otherwise(mos.rst_clip("tile","geom_clip"))
    )
)
print(f"count? {clip_df.count():,}")
clip_df.limit(10).display()

In [0]:
display(
  clip_df
    .groupBy("index_id")
      .count()
    .orderBy(F.desc("count"))
  .limit(10)
)

Let's look at one of the clipped tiles.

In [0]:
library.plot_raster(clip_df.first()["tile"]["raster"])

## Merge Rasters [H3]

> Here we show merging per H3 cell. We also write this out to delta for any follow on operations.

In [0]:
merge_h3_df = None
merge_h3_tbl_name = "band_b02_h3_merge"
if not spark.catalog.tableExists(merge_h3_tbl_name):
  (
    clip_df
      .groupBy("index_id")
        .agg(
          F.count(F.lit(1)).alias("cnt"),
          mos.rst_combineavg_agg("tile").alias("tile")
        )
      .write
        .saveAsTable(merge_h3_tbl_name)
  )
sql(f"""OPTIMIZE {merge_h3_tbl_name} ZORDER BY (index_id)""")
merge_h3_df = spark.table(merge_h3_tbl_name)
print(f"count? {merge_h3_df.count():,}")
merge_h3_df.limit(10).display()

In [0]:
library.plot_raster(merge_h3_df.first()["tile"]["raster"])

## Merge the Entire Corridor?

> A quick test of the combined size of per H3 rasters in `merge_h3_df` shows the __resulting size would be on the order of 90MB if we attempted to merge further__ - again, this can get unwieldly, so be careful. 

In [0]:
display(
  merge_h3_df
    .select(mos.rst_memsize("tile").alias("memsize"))
    .groupBy()
      .agg(
        F.sum("memsize").alias("memsum")
      )
    .withColumn("memsum_kb", (col("memsum") / F.lit(1024)))
    .withColumn("memsum_mb", (col("memsum_kb") / F.lit(1024)))
    .selectExpr("format_number(memsum_mb,0) as memsum_mb_display")
)

In [0]:
CORRIDOR_DIR = "/home/mjohns@databricks.com/stac/eo-series/corridor"
CORRIDOR_DIR_FUSE = f"/dbfs{CORRIDOR_DIR}"
os.environ['CORRIDOR_DIR'] = CORRIDOR_DIR
os.environ['CORRIDOR_DIR_FUSE'] = CORRIDOR_DIR_FUSE

dbutils.fs.mkdirs(CORRIDOR_DIR)
print(f"CORRIDOR_DIR: '{CORRIDOR_DIR}', CORRIDOR_DIR_FUSE: '{CORRIDOR_DIR_FUSE}'")

### [1] Write In-Memory Rasters

> Following from the UDF examples for Rasterio in Mosaic [docs](https://databrickslabs.github.io/mosaic/api/rasterio-udfs.html#udf-example-for-writing-raster-files-to-disk), we wil show how to write out the rasters in `merge_h3_df`. Again, this is __1,300 files totalling around 90MBs__ as-is which is not too large, and we perform the writes using a Spark UDF. Also, we add the [recommended Spark configs](https://databrickslabs.github.io/mosaic/api/rasterio-udfs.html#intro) (beyond ones already applied at top of notebook).

In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.execution.arrow.fallback.enabled", "true")

In [0]:
@udf("string")
def write_tif(raster, file_id, parent_dir):
  from pathlib import Path
  import os
  import shutil
  import tempfile
  
  Path(parent_dir).mkdir(parents=True, exist_ok=True)
  fuse_path = f"{parent_dir}/h3_{file_id}.tif"
  if not os.path.exists(fuse_path):
    with tempfile.NamedTemporaryFile() as tmpfile:
      tmpfile.write(raster)
      shutil.copyfile(tmpfile.name, fuse_path)
  return fuse_path

In [0]:
corridor_df = (
  merge_h3_df
    .select(
      "tile.raster", 
      col("index_id").alias("file_id"),
      F.lit(f"{CORRIDOR_DIR_FUSE}/band_b02").alias("parent_dir")
    )
    .repartition(128, "file_id")
    .withColumn(
      "tiff_out",
      write_tif("raster", "file_id", "parent_dir")
    )
)

out_cnt = corridor_df.groupBy("tiff_out").count().count() # <- force full execution
print(f"count? {out_cnt:,}")
# corridor_df.limit(1).display()

In [0]:
%sh
echo "num files? $(ls -1 $CORRIDOR_DIR_FUSE/band_b02 | wc -l)"
ls -lh $CORRIDOR_DIR_FUSE/band_b02 | head -5
echo "..."
ls -lh $CORRIDOR_DIR_FUSE/band_b02 | tail -5

In [0]:
library.plot_file("/dbfs/home/mjohns@databricks.com/stac/eo-series/corridor/band_b02/h3_608199668408516607.tif")

### [2] Perform Corridor Merge

> We will iterate over the persisted rasters for the corridor per each each H3 cell (from [1]) and generate a single raster. For this example, we follow steps from this [medium](https://medium.com/spatial-data-science/how-to-mosaic-merge-raster-data-in-python-fb18e44f3c8) post.

> [a] Let us first import the libraries and create the output folder with Python path lib.

In [0]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path

raster_path = Path(f'{CORRIDOR_DIR_FUSE}/band_b02')
out_dir_fuse = f'{CORRIDOR_DIR_FUSE}/out'
out_dir_local = 'out'
out_path_local = f'{out_dir_local}/mosaic_corridor_b02.tif'

Path(out_dir_fuse).mkdir(parents=True, exist_ok=True)
Path(out_dir_local).mkdir(parents=True, exist_ok=True)

print(f"raster_path: '{raster_path}', out_path_local: '{out_path_local}, out_dir_fuse? '{out_dir_fuse}'")

> [b] Now we iterate over the available .tif files in the data folder. We will merge all files in this data folder and create a mosaic from them. We also create an empty list to hold the files in the data folder.

In [0]:
raster_to_mosiac = []
raster_files = list(raster_path.iterdir())
num_files = len(raster_files)
print(f"num rasters to merge? {num_files:,}")

> [c] We then loop through the raster files, open them with rasterio and append them to the raster_to_mosiac list we created above. __Note: This took ~6 minutes running on the driver.__

In [0]:
for i,p in enumerate(raster_files):
  try:
    if (i > 0) and (i % 10 == 0):
      print(f"...{round((i/num_files) * 100, 2)}%")
    raster_to_mosiac.append(rio.open(p))
  except Exception as e:
    print(f"issue with '{p.name}'", e)
    pass

> [d] From this stage on, it is easy. We use the merge() method from rasterio to create the mosaic. We also create the output transformation parameters to use later. __Note: This took ~3 minutes running on the driver.__

In [0]:
mosaic, output = merge(raster_to_mosiac)

> [e] Now, we copy the raster's metadata and update it to match the height and width of the mosaic.

In [0]:
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": output,
    }
)
output_meta

> [f] Next, we write the mosaiced file in a local folder. __Note: The total size is 2.2G__

In [0]:
with rio.open(out_path_local, 'w', **output_meta) as m:
    m.write(mosaic)

In [0]:
%sh ls -lh out

> [g] let's render the mosaic for the corridor. __Hint: you can right click and 'open in new tab' to be able to zoom in more.__

In [0]:
# - uncomment to render from local file (note: 2.2 GBs)
# library.plot_file(out_path_local, fig_w=20, fig_h=12)

_This is a 2.2GB file. Here are screenshots of what it looks like when viewed in QGIS, showing 2 levels of zoom._

In [0]:
displayHTML("""<img src="..." width="100%">""")

In [0]:
displayHTML("""<img src="..." width="100%">""")

> [h] once satisfied, we can copy up to out_dir_fuse.

In [0]:
%sh
cp out/mosaic_corridor_b02.tif $CORRIDOR_DIR_FUSE/out
ls -lh $CORRIDOR_DIR_FUSE/out

In [0]:
# - this is faster from the local file
#library.plot_file(f"{CORRIDOR_DIR_FUSE}/out/mosaic_corridor_b02.tif", fig_w=16, fig_h=16)